<a href="https://colab.research.google.com/github/HuyenNguyenHelen/TREC-PM-2020/blob/master/BERT_Innitial_Reranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U sentence-transformers rank_bm25

Requirement already up-to-date: sentence-transformers in /usr/local/lib/python3.7/dist-packages (2.0.0)
Requirement already up-to-date: rank_bm25 in /usr/local/lib/python3.7/dist-packages (0.2.1)


# Documents and Query processing

### Documents

In [4]:
import pandas as pd
path = r'/content/New_Summary_Combined_NER_BoolQuery_scores_2016_1.csv'
with open(path, 'r', encoding = 'utf-8' ) as f:
    data = pd.read_csv(f, header = None, delimiter='\t')
data.columns = ['queryID','docID', 'score', 'rank', 'concat_str']
data.head()

# column_names = [name.split(':')[0].replace('\'', '') for name in data.columns]
# column_names = [name.split(':')[0].replace(' ', '') for name in column_names]
# data.columns = column_names
# print(data.columns )

# # Creating new data with some txt fields
# new_data = data[['brief_title', 'official_title', 'brief_summary_text_block', 'concat_string']]
# new_data['brief_summary_text_block'] = new_data['brief_summary_text_block'].apply(lambda x: x.split(':')[1].replace("'",''))
# new_data['brief_summary_text_block'].head()

,queryID,docID,score,rank,concat_str
0,1,4529650,15.569978,1.000,duoden adenocarcinoma melena duoden melena mic...
1,1,4798841,15.293642,0.999,preoper cavern hemangioma present melena wirel...
2,1,3999026,15.052819,0.998,anemia melena pyeloduoden fistula stoneassoci ...
3,1,3113951,14.672893,0.997,ultrasonograph cow abomas lymphosarcoma descri...
4,1,3132483,14.494692,0.996,cecal schwannoma gastrointestin bleed woman sc...


In [5]:
doc_txt = data['concat_str'].values.tolist()

### Query

In [6]:
# Processed query 2016
path = r"/content/Quries_2016.txt"
with open(path, 'r', encoding = 'utf-8' ) as f:
    query_df = pd.read_csv(f, delimiter='\t')
query_df.columns = ['QueryID', 'NERgdd']
query_df['new_NERgdd'] = query_df['NERgdd'].apply(lambda x: ' '.join([i for i in str(x).split(',')]))

query_df.head()


,QueryID,NERgdd,new_NERgdd
0,2,"right,hip,arthroplasti",right hip arthroplasti
1,3,"75f,hypoglycem",75f hypoglycem
2,4,osteoporosi,osteoporosi
3,5,"cough,product",cough product
4,6,"atrial,pe,fibril",atrial pe fibril


In [7]:
# Processing query
queries = query_df['new_NERgdd'].values.tolist()

# Question-Answering using Simple Wikipedia Index

This examples demonstrates the setup for Query / Question-Answer-Retrieval.

You can input a query or a question. The script then uses semantic search
to find relevant passages in Simple English Wikipedia (as it is smaller and fits better in RAM).

For semantic search, we use SentenceTransformer('msmarco-distilbert-base-v2') and retrieve
100 potentially passages that answer the input query.

Next, we use a more powerful CrossEncoder (cross_encoder = CrossEncoder('cross-encoder/ms-marco-TinyBERT-L-6')) that
scores the query and all retrieved passages for their relevancy. The cross-encoder is neccessary to filter out certain noise
that might be retrieved from the semantic search step.


In [8]:
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import time
import gzip
import os
import torch

if not torch.cuda.is_available():
    print("Warning: No GPU found. Please add GPU to your notebook")

# We use the Bi-Encoder to encode all passages, so that we can use it with sematic search
model_name = 'msmarco-MiniLM-L-6-v3'
bi_encoder = SentenceTransformer(model_name)
top_k = 1000  # Number of passages we want to retrieve with the bi-encoder

# The bi-encoder will retrieve 100 documents. We use a cross-encoder, to re-rank the results list to improve the quality
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
        
# Here, we compute the corpus_embeddings from scratch (which can take a while depending on the GPU)
corpus_embeddings = bi_encoder.encode(doc_txt, convert_to_tensor=True, show_progress_bar=True)
print(corpus_embeddings)


tensor([[ 0.0024, -0.2993,  0.0318,  ..., -0.3888, -0.2327,  0.1130],
        [ 0.0950,  0.0926,  0.4330,  ..., -0.4781,  0.3871,  0.3569],
        [ 0.2041, -0.1071, -0.0122,  ..., -0.1918,  0.2932,  0.3275],
        ...,
        [ 0.2338,  0.6296,  0.3718,  ...,  0.1604,  0.1702,  0.7025],
        [-0.1274,  0.0692,  0.3458,  ..., -0.1609,  0.2653,  0.5241],
        [-0.8699,  0.1260,  0.0257,  ..., -0.3380,  0.2821,  0.5499]],
       device='cuda:0')


In [9]:
txt = 'avail urea nitrogencreatinin gastrointestin bleed melena aim urea nitrogen creatinin buncr distinguish upper gastrointestin bleed gib differenti upper gastrointestin bleed ugib present melena retrospect gib present melena review demograph modal transfus ugib gastrointestin bleed lgib 583 233 ugib varic 371 peptic ulcer pud 314 buncr ugib lgib odd 69 confid interv 13372 ugib buncr varic pud 0015 buncr appear unit varic pud 10313 pud meckel diverticulum buncr uneven differenti ugib lgib melena buncr interpret care gastrointestinalbleedingmelenachild introductionmateri methodsresultsclin enrol patientsclin ugib lgib melena patientsdiagnost modal subjectslaboratori melena ugib lgiblaboratori melena causesr transfus gib melenadiscuss melena refer tarri black stool gastrointestin pediatr melena upper gastrointestin bleed ugib ligament tretiz intestin proxim colon bleed melena slow brisk bleed 50100 ml bleed stomach continu bleed stop fast massiv bleed hematochezia advanc bleed tarri black stool highrat bleed esophag varic duoden ulcer dieulafoy arterioven malform meckel diverticulum melena esophag varic peptic ulcer pud meckel diverticulum bleed hemodynam stabil priorit bleed proper initi bleed hematemesi melena abdomin locat gastrointestin gastrointestin lavag present lavag esophag bleed give discomfort yield duoden bleed urea nitrogen creatinin buncr differenti ugib gastrointestin bleed lgib urea nitrogen gastrointestin bleed absorpt intestin excret urea nitrogen hypovolem buncr impli ugib pud bleed melena bleed bun insignific aforement research gib differenti diver bleed avail buncr distinguish ugib lgib caus ugib melena buncr distinguish ugib lgib help local bleed gib present melena melena buncr buncr differenti ugib lgib gib present melena buncr 69 ugib lgib buncr manifest ugib varic pud pud ugib meckel diverticulum lgib buncr local bleed gib felber buncr ugib ugib lgib gib ugib buncr 98 688 buncr ratio ugib lgib lgib buncr 486 ugib 143 lgib 125 gib buncr 36 impli ugib bleed hardli local pediatr age birth 180 buncr rang 108 124 median 320 ugib 104 354 median 225 lgib use buncr buncr differenti ugib lgib buncr pud rang 135 381 median varic rang 176 124 median 400 ugib melena buncr differ ugib buncr pud attribut zero buncr melena gib pud buncr meckel diverticulum buncr pud meckel diverticulum cleveland age birth old ugib underw egd prolaps gastropathi 127 erosionsulc 108 eros esophag duoden erosionsulc 82 esophag varic 63 ugib gastritiserosionulc 358 duodenitisduoden ulcer 76 esophag varic 72 hassoon ugib esophag varic 39 erosionulc 293 duoden ulcer 74 esophag 49 ugib esophagogastr varic 351 gastritisgastr ulcer 162 duodenitisduoden ulcer 216 esophag 54 varic bleed hassoon lgib account 226 melena melena meckel diverticulum gib melena differ hospit gib 1115 lowest younger old ugib induc melena 297 270 216 month 135 81 610 month lgib 429 610 meckel diverticulum rafeey esophag varic bleed 04 younger 67 esophag varic rare younger old varic bleed age 43 youngest varic bleed portal vein agenesi varic bleed lymphocyt count pud meckel diverticulum influenc preced fever cough 538 varic bleed gib melena custom pud antacid except duoden ulcer bleed endoscop endoscop varic ligat histoacryl inject varic bleed hemoclip treat dieulafoy lesion snare polypectomi colon polyp meckel diverticulum duoden ulcer arterioven malform bowel arterioven malform gastrointestin stromal huge duoden polyp gib melena stop spontan sometim intens repetit continu lifethreaten consid hemodynam reserv younger ass restor hemodynam transfus improv deterior perfus enhanc suppli lifesav restrict transfus ugib gdl hemoglobin liber transfus gdl sharma gib refus bloodderiv ethic religi concern transfusionfre safe refus transfus nevertheless 18yearold appli hemodynam reserv transfus done 517 501 ugib varic bleed 889 lgib meckel diverticulum melena melena underw transfus melena statist insignific 182 pud underw transfus varic 692 meckel diverticulum 667 transfus 571 hemoglobin gdl belong restrict retrospect undiagnos possibl bia buncr classifi buncr buncr prove distinguish bleed ugib lgib gib melena buncr appear differ uneven ugib varic pud pud insignificantli lgib meckel diverticulum buncr appear varic pud pud mickel diverticulum impli ugib confound lgib melena depend transit gastrointestin tract duoden bleed observ interpret appli buncr distinguish ugib lgib largescal one'
test = data[data['concat_str']==txt]['docID'].values
type(test.astype(str))
test[0]

4391998

In [13]:
##### Sematic Search
all_output = []
for id, query in zip(query_df['QueryID'], queries):
    
  # Encode the query using the bi-encoder and find potentially relevant passages
    question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    question_embedding = question_embedding.cuda()
    hits_en = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
    for hit in hits_en:
      tempt_dic = []
      output = {}
      #hit = sorted(hits, key=lambda x: x['score'], reverse=True)
      for h in hit:   
        #temp_dict = {'doc': [], 'score': []}
        tempt_dic.append([data[data['concat_str']==doc_txt[h['corpus_id']]]['docID'].values[0], h['score']] )
        with open('/content/semantic_output.txt', "a") as outFile:
            outFile.write(str(id) + "\t" + str(data[data['concat_str']==doc_txt[h['corpus_id']]]['docID'].values[0]) + '\t' + str(h['score']) + "\n")
      output[id] = tempt_dic
      
      
    all_output.append(output)


all_output[0]

{2: [[2361975, 0.5958926677703857],
  [4753887, 0.5712393522262573],
  [2504677, 0.5501634478569031],
  [2426685, 0.5193310976028442],
  [3229990, 0.497448593378067],
  [2432021, 0.495950847864151],
  [3597125, 0.4945206642150879],
  [3257425, 0.4937623143196106],
  [4137521, 0.4928174614906311],
  [3132062, 0.4897884130477905],
  [3232250, 0.4890596270561218],
  [3421930, 0.48434293270111084],
  [4544332, 0.481203556060791],
  [3032109, 0.47906258702278137],
  [2762554, 0.474350243806839],
  [2981773, 0.4708772301673889],
  [4595123, 0.4689294099807739],
  [4775489, 0.46863478422164917],
  [3214740, 0.4679725170135498],
  [4545570, 0.4664550721645355],
  [2856205, 0.4658034145832062],
  [2826290, 0.46269842982292175],
  [2911928, 0.45929932594299316],
  [4462442, 0.45424243807792664],
  [3369145, 0.454234778881073],
  [3337746, 0.45274239778518677],
  [2703995, 0.4524977207183838],
  [3767903, 0.4513447880744934],
  [4435661, 0.45110687613487244],
  [3768049, 0.44920867681503296],
  [

In [11]:
#### Cross-encoding retrieval

for id, query in zip(query_df['QueryID'], queries):

    ##### Sematic Search #####
    # Encode the query using the bi-encoder and find potentially relevant passages
    question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    question_embedding = question_embedding.cuda()
    hits_cross = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
    for hit in hits_cross:
      ##### Re-Ranking #####
      # Now, score all retrieved passages with the cross_encoder
      #print(hit)
      #print([doc_txt[h['corpus_id']] for h in hit])
      cross_inp = [[query, doc_txt[h['corpus_id']]] for h in hit]
      cross_scores = cross_encoder.predict(cross_inp)

      # Sort results by the cross-encoder scores
      for idx in range(len(cross_scores)):
          hit[idx]['cross-score'] = cross_scores[idx]
      tempt_dic = []
      output = {}
      #hit = sorted(hits, key=lambda x: x['score'], reverse=True)
      for h in hit:   
        #temp_dict = {'doc': [], 'score': []}
        tempt_dic.append([data[data['concat_str']==doc_txt[h['corpus_id']]]['docID'].values[0], h['score']] )
        with open('/content/rerank_cross_output.txt', "a") as outFile:
            outFile.write(str(id) + "\t" + str(data[data['concat_str']==doc_txt[h['corpus_id']]]['docID'].values[0]) + '\t' + str(h['score']) + "\n")
      output[id] = tempt_dic
      

In [ ]:
# This function will search all wikipedia articles for passages that
# answer the query
def search(query):

    ##### Sematic Search #####
    # Encode the query using the bi-encoder and find potentially relevant passages
    question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    question_embedding = question_embedding.cuda()
    hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
    hits = hits[0]  # Get the hits for the first query

    ##### Re-Ranking #####
    # Now, score all retrieved passages with the cross_encoder
    cross_inp = [[query, doc_txt[hit['corpus_id']]] for hit in hits]
    cross_scores = cross_encoder.predict(cross_inp)

    # Sort results by the cross-encoder scores
    for idx in range(len(cross_scores)):
        hits[idx]['cross-score'] = cross_scores[idx]

    # Output of top-5 hits from bi-encoder
    print("\n-------------------------\n")
    print("Top-3 Bi-Encoder Retrieval hits")
    hits_en = sorted(hits, key=lambda x: x['score'], reverse=True)
    for hit in hits_en[0:3]:
         print("\t{:.3f}\t{}".format(hit['score'], doc_txt[hit['corpus_id']].replace("\n", " ")))

    # Output of top-5 hits from re-ranker
    print("\n-------------------------\n")
    print("Top-3 Cross-Encoder Re-ranker hits")
    hits_re = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
    for hit in hits_re[0:3]:
        print("\t{:.3f}\t{}".format(hit['cross-score'], doc_txt[hit['corpus_id']].replace("\n", " ")))
    #return hits_re['cross-score'], doc_txt[int(hits_re['corpus_id'])]




